# 2025년 10월 1일~5일 SNS_COMMUNITY DATA ANALYSIS

## (1) SNS 데이터 워드클라우드

### A. 필요 라이브러리 임포트

In [2]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
import re

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

### B. 데이터 로드 및 필터링

In [6]:
# 데이터 로드
df = pd.read_csv('./FINAL_SNS_YOUTUBE.csv', encoding='utf-8-sig')

print(f"전체 데이터: {len(df):,}행")
print(f"날짜 범위: {df['date'].min()} ~ {df['date'].max()}")
print(f"\n컬럼: {df.columns.tolist()}")

전체 데이터: 6,382행
날짜 범위: 20250901 ~ 20251031

컬럼: ['date', 'platform', 'type', 'content', 'engagement', 'author', 'url', 'original_date', 'id']


In [10]:
# 2025년 10월 1일~5일 데이터 필터링
start_date = 20251001
end_date = 20251005
df_oct5 = df[(df['date'] >= start_date) & (df['date'] <= end_date)].copy()

print(f"\n{'='*100}")
print(f"2025년 10월 1일~5일 SNS데이터")
print(f"{'='*100}")
print(f"SNS데이터수: {len(df_oct5):,}개")
print(f"\n샘플데이터:")
for i, content in enumerate(df_oct5['content'], 1):
    print(f"{i:2d}. {content}")


2025년 10월 1일~5일 SNS데이터
SNS데이터수: 80개

샘플데이터:
 1. Market Edge ANC 1 October 2025 Heres the news from Market Edge this October 1 2025 Headlines Asian Factory Activity Roundup Philippine Corporate News
 2. 03 October 2025 Market Analysis shorts Nifty showed a strong bullish move today As we shared earlier a Long trade following the market trend could have been taken
 3. Super Red Trade Recovery How I Avoid Panic Selling Near My Entry The Nightmare Scenario Down 3000 on a Single Trade Every trader has been there Youre in a super red trade perhaps
 4. Monthly Market Outlook October 2025 by Prateek Agrawal In this video we share our perspective on the Q1 FY25 results and the key market trends shaping the investment landscape
 5. All Vortex Blue Bot bot trading gather here LOL
 6. 100 leverage 2980 margin on a 298K size would put a shorts liq just above entry within 12 if isolated 2925 is 31 below entryimpossible for that short with that margin
 7. The central bank is addressing the inflation

### C. 데이터 로드 및 필터링

In [ ]:
# 모든 content를 하나의 문자열로 결합
all_titles = ' '.join(df_oct5['title'].dropna().astype(str))

print(f"전체 텍스트 길이: {len(all_titles):,}자")
print(f"\n전체 텍스트 샘플 (처음 500자):")
print(all_titles[:500])